In [1]:
import pyximport; pyximport.install()
from gpcode import *
from csvparser import *

import math

In [2]:
def handMapToVector(handResults_dict):
    result = []
    
    mapping = {'raise': 1.0, 'check': 0.5, 'fold': 0.0}
    
    keys = handResults_dict.keys()
    keys = sorted(keys)
    
    sub_keys = handResults_dict[0].keys()
    sub_keys = sorted(sub_keys)
    
    for i in keys:
        result.append([])
        for j in range(0,len(sub_keys)):
            data = handResults_dict[i][sub_keys[j]]
            
            result[i].append(mapping[data])
    
    return result

In [10]:
def distance(v1, v2):
    result = 0.0
    
    for i in range(len(v1)):
        #temp = 0.0
        for j in range(len(v1[i])):
            if v2[i][j] != v1[i][j]:
                result += 1.0
            #result += (v2[i][j] - v1[i][j]) ** 2
    
    #return math.sqrt(result)
    return result

In [11]:
#ind = gp.PrimitiveTree.from_string("'raise'", pset)
#ind2 = gp.PrimitiveTree.from_string("'check'", pset)
#ind3 = gp.PrimitiveTree.from_string("'fold'", pset)
#i = handMapToVector(mapHandsToResults(ind, all_hands))
#j = handMapToVector(mapHandsToResults(ind2, all_hands))
#k = handMapToVector(mapHandsToResults(ind3, all_hands))

45.9619407771
91.9238815543
45.9619407771


In [11]:
csv_filename = "Comparison Results/mapping-" + machine_alias + ".csv.txt"

fields = [['Heuristic'], ['Complexity'], ['Fitness']]
conditions = []

csv_data = extractDictFromCSV(csv_filename)
parsed_data = extractIndividualsFromData(csv_data, ind_field=fields[0], complexity_field= fields[1], fitness_field=fields[2], condition=conditions)
parsed_data = sorted(parsed_data, reverse=True)

In [13]:
results = {}

csv_filename = "Distance Results/mapping-" + machine_alias + "-" + str(datetime.datetime.now()) + ".csv.txt"
if not os.path.isfile(csv_filename):
    createLogFile(csv_filename, "Heuristic1; Heuristic2; Distance;\n")

c = 0
print c
for x in parsed_data:
    results[x.heuristic] = 0.
    ind1 = gp.PrimitiveTree.from_string(x.heuristic, pset)
    v1 = handMapToVector(mapHandsToResults(ind1, all_hands))

    for y in parsed_data:
        if x.heuristic != y.heuristic:
            ind2 = gp.PrimitiveTree.from_string(y.heuristic, pset)

            v2 = handMapToVector(mapHandsToResults(ind2, all_hands))
            
            d = distance(v1, v2)

            logToFile(csv_filename, str(ind1) + "; " + str(ind2) + "; "  + str(d) + ";\n")
            results[x.heuristic] += d
    c += 1
    print c

print results

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


NameError: name 'result' is not defined